# Fetching Data from Oanda API Endpoints

Oanda API endpoints can be found at http://developer.oanda.com/rest-live-v20/account-ep/

In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
account_data = pd.read_csv("~/Documents/temp/oanda/testnet_key.txt", sep=" ", header=None)
API_KEY = account_data[0][2]
ACCOUNT_ID = account_data[0][1]
OANDA_URL = account_data[0][0]

In [3]:
session = requests.Session()

## Fetching Data from the Candles Endpoint

In [4]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [6]:
def set_candles_url(url, pair):
 return f"{url}/instruments/{pair}/candles"

In [7]:
pair_name = "EUR_USD"
url_candles = set_candles_url(OANDA_URL, pair_name)

In [8]:
response = session.get(url_candles, params=params, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 3501,
   'time': '2022-07-21T03:00:00.000000000Z',
   'bid': {'o': '1.02088', 'h': '1.02127', 'l': '1.02036', 'c': '1.02083'},
   'mid': {'o': '1.02094', 'h': '1.02136', 'l': '1.02044', 'c': '1.02091'},
   'ask': {'o': '1.02100', 'h': '1.02145', 'l': '1.02051', 'c': '1.02099'}},
  {'complete': True,
   'volume': 3606,
   'time': '2022-07-21T04:00:00.000000000Z',
   'bid': {'o': '1.02084', 'h': '1.02247', 'l': '1.02048', 'c': '1.02223'},
   'mid': {'o': '1.02091', 'h': '1.02255', 'l': '1.02056', 'c': '1.02231'},
   'ask': {'o': '1.02098', 'h': '1.02264', 'l': '1.02062', 'c': '1.02239'}},
  {'complete': True,
   'volume': 3922,
   'time': '2022-07-21T05:00:00.000000000Z',
   'bid': {'o': '1.02221', 'h': '1.02276', 'l': '1.02100', 'c': '1.02222'},
   'mid': {'o': '1.02229', 'h': '1.02285', 'l': '1.02108', 'c': '1.02230'},
   'ask': {'o': '1.02237', 'h': '1.02294', 'l': '1.02115', 'c': '1.02239'}},

## Fetching Data from the Instruments Endpoint

In [11]:
url_inst = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [12]:
response = session.get(url_inst, params=params, data=None, headers=None)

In [13]:
response.status_code

200

In [14]:
data = response.json() # dict_keys(['instruments', 'lastTransactionID'])

In [15]:
instruments_list = data["instruments"]

In [16]:
len(instruments_list)

129

In [17]:
instruments_list

[{'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'minimumTradeSize': '0.1',
  'maximumTrailingStopDistance': '10000.0',
  'minimumTrailingStopDistance': '5.0',
  'maximumPositionSize': '0',
  'maximumOrderUnits': '500',
  'marginRate': '0.1',
  'guaranteedStopLossOrderMode': 'DISABLED',
  'tags': [{'type': 'ASSET_CLASS', 'name': 'INDEX'}],
  'financing': {'longRate': '-0.0258',
   'shortRate': '-0.0242',
   'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
    {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
    {'dayOfWeek': 'FRIDAY', 'daysCharged': 3},
    {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
    {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
 {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'trade

In [18]:
# taking a look at the information that's available for every instrument
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [19]:
# list of keys we're intersted in getting 
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 
         'tradeUnitsPrecision', 'marginRate']

In [20]:
instruments_dict = {}
for i in instruments_list:
    # print(i['name'])
    key = i['name']
    instruments_dict[key] = {
        k: i[k] for k in key_i
    }

In [21]:
instruments_dict

{'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'AU200_AUD': {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'GBP_AUD': {'name': 'GBP_AUD',
  'type': 'CURRENCY',
  'displayName': 'GBP/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'SG30_SGD': {'name': 'SG30_SGD',
  'type': 'CFD',
  'displayName': 'Singapore 30',
  'pipLocation': -1,
  'displayPrecision': 2,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  

In [22]:
# fetching particular instrument
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [23]:
with open("Data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

## Fetching a candle pair

In [38]:
def fetch_candles(api_url, pair_name, count="10", granularity="H1"):
    url = f"{api_url}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    status = response.status_code
    
    if status == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return status, data

In [25]:
status, data = fetch_candles(OANDA_URL, "EUR_USD", count=20)

In [26]:
status

200

In [27]:
len(data)

20

In [28]:
data

[{'complete': True,
  'volume': 6741,
  'time': '2022-07-20T17:00:00.000000000Z',
  'bid': {'o': '1.01898', 'h': '1.01924', 'l': '1.01547', 'c': '1.01570'},
  'mid': {'o': '1.01905', 'h': '1.01932', 'l': '1.01554', 'c': '1.01578'},
  'ask': {'o': '1.01912', 'h': '1.01941', 'l': '1.01562', 'c': '1.01586'}},
 {'complete': True,
  'volume': 4722,
  'time': '2022-07-20T18:00:00.000000000Z',
  'bid': {'o': '1.01568', 'h': '1.01768', 'l': '1.01565', 'c': '1.01766'},
  'mid': {'o': '1.01576', 'h': '1.01776', 'l': '1.01573', 'c': '1.01774'},
  'ask': {'o': '1.01584', 'h': '1.01783', 'l': '1.01580', 'c': '1.01782'}},
 {'complete': True,
  'volume': 2846,
  'time': '2022-07-20T19:00:00.000000000Z',
  'bid': {'o': '1.01767', 'h': '1.01834', 'l': '1.01733', 'c': '1.01769'},
  'mid': {'o': '1.01774', 'h': '1.01841', 'l': '1.01741', 'c': '1.01776'},
  'ask': {'o': '1.01781', 'h': '1.01848', 'l': '1.01748', 'c': '1.01783'}},
 {'complete': True,
  'volume': 1583,
  'time': '2022-07-20T20:00:00.0000000

In [29]:
# getting the prices from the JSON object
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
# loop through the prices and make for each one a column key 
# that looks like mid_c, mid_o, mid_h, mid_c
for p in prices:
    for o in ohlc:
        print(f"{p}_{o}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [30]:
# collecting certain info from the instruments
# and converting it to a data frame
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []
    
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        # loop through the prices and make for each one a column key 
        # that looks like mid_c, mid_o, mid_h, mid_c
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

In [31]:
df = get_candles_df(data)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    19 non-null     datetime64[ns, tzutc()]
 1   volume  19 non-null     int64                  
 2   mid_o   19 non-null     float64                
 3   mid_h   19 non-null     float64                
 4   mid_l   19 non-null     float64                
 5   mid_c   19 non-null     float64                
 6   bid_o   19 non-null     float64                
 7   bid_h   19 non-null     float64                
 8   bid_l   19 non-null     float64                
 9   bid_c   19 non-null     float64                
 10  ask_o   19 non-null     float64                
 11  ask_h   19 non-null     float64                
 12  ask_l   19 non-null     float64                
 13  ask_c   19 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), 

In [33]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-20 17:00:00+00:00,6741,1.01905,1.01932,1.01554,1.01578,1.01898,1.01924,1.01547,1.01570,1.01912,1.01941,1.01562,1.01586
1,2022-07-20 18:00:00+00:00,4722,1.01576,1.01776,1.01573,1.01774,1.01568,1.01768,1.01565,1.01766,1.01584,1.01783,1.01580,1.01782
2,2022-07-20 19:00:00+00:00,2846,1.01774,1.01841,1.01741,1.01776,1.01767,1.01834,1.01733,1.01769,1.01781,1.01848,1.01748,1.01783
3,2022-07-20 20:00:00+00:00,1583,1.01776,1.01816,1.01741,1.01810,1.01769,1.01806,1.01732,1.01800,1.01784,1.01826,1.01749,1.01821
4,2022-07-20 21:00:00+00:00,310,1.01784,1.01812,1.01774,1.01796,1.01747,1.01797,1.01745,1.01780,1.01822,1.01838,1.01793,1.01813
5,2022-07-20 22:00:00+00:00,1536,1.01798,1.01876,1.01784,1.01818,1.01778,1.01867,1.01768,1.01809,1.01819,1.01886,1.01798,1.01826
6,2022-07-20 23:00:00+00:00,1909,1.01820,1.01849,1.01768,1.01810,1.01811,1.01840,1.01759,1.01802,1.01828,1.01858,1.01777,1.01818
7,2022-07-21 00:00:00+00:00,3609,1.01810,1.01936,1.01784,1.01870,1.01802,1.01928,1.01776,1.01863,1.01817,1.01945,1.01791,1.01878
8,2022-07-21 01:00:00+00:00,4698,1.01874,1.01950,1.01816,1.01940,1.01866,1.01941,1.01808,1.01933,1.01881,1.01959,1.01824,1.01948
9,2022-07-21 02:00:00+00:00,3117,1.01939,1.02104,1.01914,1.02094,1.01932,1.02096,1.01906,1.02087,1.01946,1.02111,1.01921,1.02101


In [34]:
# testing the code once again
status, data = fetch_candles(OANDA_URL, "GBP_AUD", count=10, granularity="H4")
candles_df = get_candles_df(data)
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-19 21:00:00+00:00,14876,1.73993,1.74132,1.73816,1.73991,1.73867,1.74110,1.73791,1.73975,1.74119,1.74233,1.73837,1.74007
1,2022-07-20 01:00:00+00:00,25017,1.73984,1.74265,1.73711,1.73802,1.73962,1.74244,1.73691,1.73785,1.74007,1.74286,1.73730,1.73820
2,2022-07-20 05:00:00+00:00,45293,1.73804,1.74088,1.73599,1.73823,1.73786,1.74059,1.73582,1.73807,1.73823,1.74122,1.73615,1.73839
3,2022-07-20 09:00:00+00:00,53174,1.73821,1.74005,1.73290,1.73586,1.73804,1.73988,1.73277,1.73571,1.73838,1.74022,1.73304,1.73602
4,2022-07-20 13:00:00+00:00,55805,1.73591,1.74048,1.73498,1.73932,1.73577,1.74033,1.73483,1.73913,1.73605,1.74063,1.73511,1.73950
5,2022-07-20 17:00:00+00:00,28043,1.73928,1.74026,1.73748,1.73902,1.73911,1.74008,1.73731,1.73881,1.73944,1.74044,1.73764,1.73922
6,2022-07-20 21:00:00+00:00,13224,1.73897,1.74004,1.73625,1.73722,1.73822,1.73984,1.73608,1.73701,1.73972,1.74066,1.73642,1.73744
7,2022-07-21 01:00:00+00:00,29966,1.73730,1.73985,1.73532,1.73542,1.73708,1.73966,1.73512,1.73522,1.73751,1.74004,1.73550,1.73561
8,2022-07-21 05:00:00+00:00,48473,1.73540,1.73962,1.73504,1.73843,1.73521,1.73947,1.73485,1.73827,1.73558,1.73980,1.73522,1.73859


In [35]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    9 non-null      datetime64[ns, tzutc()]
 1   volume  9 non-null      int64                  
 2   mid_o   9 non-null      float64                
 3   mid_h   9 non-null      float64                
 4   mid_l   9 non-null      float64                
 5   mid_c   9 non-null      float64                
 6   bid_o   9 non-null      float64                
 7   bid_h   9 non-null      float64                
 8   bid_l   9 non-null      float64                
 9   bid_c   9 non-null      float64                
 10  ask_o   9 non-null      float64                
 11  ask_h   9 non-null      float64                
 12  ask_l   9 non-null      float64                
 13  ask_c   9 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in

## Saving the fetched Candles in a Data File

In [36]:
def create_data_file(api_url, pair_name, count="10", granularity="H1"):
    
    status, data = fetch_candles(
        api_url, 
        pair_name, 
        count=count, 
        granularity=granularity
    )
    
    if status != 200:
        print("Failed", pair_name, data)
        return    
    
    if len(data) == 0:
        print("No candles", pair_name)
        return
    
    candles_df = get_candles_df(data)
    # safe compressed data frame to file
    candles_df.to_pickle(f"Data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [37]:
create_data_file(OANDA_URL, "USD_PLN", count=10, granularity="H4")

USD_PLN H4 9 candles, 2022-07-19 21:00:00+00:00 2022-07-21 05:00:00+00:00
